In [1]:
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.0 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29880 sha256=53c63e2ad76b843966290210aab081ea904ab2b4bc5330f4662d4f3129466e5d
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=04b3931a82dfed34c2b43d1312015f114044d86bb12c2952bc844dddee1638e1
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [43]:
import pandas as pd
from fancyimpute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy.sparse import csr_matrix

In [44]:
# show all columns
pd.options.display.max_columns = None

In [45]:
df_train = pd.read_csv('new_epl_training.csv')
df_train.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,EPL,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter
0,2000-08-19,Charlton,Man City,4,0.0,H,2,0,H,Rob Harris,17,8,14,4,6,6,13,12,1,2,0,0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1,-1,0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,1,0,0,0,0,0
1,2000-08-19,Chelsea,West Ham,4,2.0,H,1,0,H,Graham Barber,17,12,10,5,7,7,19,14,1,2,0,0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5,-1,0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,1,0,0,0,0,0
2,2000-08-19,Coventry,Middlesbrough,1,3.0,A,1,1,D,Barry Knight,6,16,3,9,8,4,15,21,5,3,1,0,18.0,11.4,14.4,0.9,0.0,17.7,2000-2001,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6,2,1,-1.000000,1500.0,1500.0,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,1,0,0,0,0,0
3,2000-08-19,Derby,Southampton,2,2.0,D,1,2,A,Andy D'Urso,6,13,4,6,5,8,11,13,1,1,0,0,17.7,9.3,13.7,2.8,0.0,18.6,2000-2001,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2,0,0,-0.666667,1500.0,1500.0,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,1,0,0,0,0,0
4,2000-08-19,Leeds,Everton,2,0.0,H,2,0,H,Dermot Gallagher,17,12,8,6,6,4,21,20,1,3,0,0,18.0,10.1,13.6,4.0,0.0,20.8,2000-2001,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1,-2,0,-0.333333,1500.0,1500.0,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,1,0,0,0,0,0


In [46]:
df_test = pd.read_csv('epl-test.csv')
df_test

,Date,HomeTeam,AwayTeam
0,03-Feb-24,AFC Bournemouth,Nottingham Forest
1,03-Feb-24,Arsenal,Liverpool
2,03-Feb-24,Brentford,Man City
3,03-Feb-24,Brighton,Crystal Palace
4,03-Feb-24,Burnley,Fulham
5,03-Feb-24,Chelsea,Wolves
6,03-Feb-24,Everton,Spurs
7,03-Feb-24,Man Utd,West Ham
8,03-Feb-24,Newcastle,Luton Town
9,03-Feb-24,Sheff Utd,Aston Villa


In [47]:
# make test file consistent with train file
team_names_changes = {
    'AFC Bournemouth': 'Bournemouth',
    'Nottingham Forest': "Nott'm Forest",
    'Spurs': 'Tottenham',
    'Man Utd': 'Man United',
    'Luton Town': 'Luton',
    'Sheff Utd': 'Sheffield United'
}

df_test['HomeTeam'] = df_test['HomeTeam'].replace(team_names_changes)
df_test['AwayTeam'] = df_test['AwayTeam'].replace(team_names_changes)

df_test

,Date,HomeTeam,AwayTeam
0,03-Feb-24,Bournemouth,Nott'm Forest
1,03-Feb-24,Arsenal,Liverpool
2,03-Feb-24,Brentford,Man City
3,03-Feb-24,Brighton,Crystal Palace
4,03-Feb-24,Burnley,Fulham
5,03-Feb-24,Chelsea,Wolves
6,03-Feb-24,Everton,Tottenham
7,03-Feb-24,Man United,West Ham
8,03-Feb-24,Newcastle,Luton
9,03-Feb-24,Sheffield United,Aston Villa


In [65]:
df_combined = pd.concat([df_train, df_test], ignore_index=True)
df_combined

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,EPL,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter
0,2000-08-19,Charlton,Man City,4.0,0.0,H,2.0,0.0,H,Rob Harris,17.0,8.0,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.0,-1.0,0.0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,1.0,0.0,0.0,0.0,0.0,0.0
1,2000-08-19,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H,Graham Barber,17.0,12.0,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,2000-2001,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5.0,-1.0,0.0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,1.0,0.0,0.0,0.0,0.0,0.0
2,2000-08-19,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D,Barry Knight,6.0,16.0,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0,18.0,11.4,14.4,0.9,0.0,17.7,2000-2001,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6.0,2.0,1.0,-1.000000,1500.0,1500.0,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,1.0,0.0,0.0,0.0,0.0,0.0
3,2000-08-19,Derby,Southampton,2.0,2.0,D,1.0,2.0,A,Andy D'Urso,6.0,13.0,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0,17.7,9.3,13.7,2.8,0.0,18.6,2000-2001,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2.0,0.0,0.0,-0.666667,1500.0,1500.0,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,1.0,0.0,0.0,0.0,0.0,0.0
4,2000-08-19,Leeds,Everton,2.0,0.0,H,2.0,0.0,H,Dermot Gallagher,17.0,12.0,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0,18.0,10.1,13.6,4.0,0.0,20.8,2000-2001,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1.0,-2.0,0.0,-0.333333,1500.0,1500.0,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,03-Feb-24,Chelsea,Wo

In [49]:
df_combined.drop(['Referee', 'FTHG', 'FTAG', 'HTR'], axis=1, inplace=True)

In [50]:
# problems with cup games? all ftr are 0. drop them for now
df_combined = df_combined[df_combined['EPL'] != 0]

In [51]:
df_combined['Date'] = pd.to_datetime(df_combined['Date'])
df_combined['Year'] = df_combined['Date'].dt.year
df_combined['Month'] = df_combined['Date'].dt.month
df_combined['Day'] = df_combined['Date'].dt.day
df_combined.drop('Date', axis=1, inplace=True)

In [52]:
# put data that we already know into the test dataset
df_combined['Season'].fillna('2023-2024', inplace=True)
df_combined['EPL'].fillna(1, inplace=True)

In [53]:
# change season to sequential numbers
seasons = sorted(df_combined['Season'].unique())
sequential_season = {season: i+1 for i, season in enumerate(seasons)}
df_combined['Season'] = df_combined['Season'].map(sequential_season)
df_combined

,HomeTeam,AwayTeam,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,temperature_2m_max,temperature_2m_min,temperature_2m_mean,rain_sum (mm),snowfall_sum (cm),wind_speed_10m_max (km/h),Season,Home_GK_Ability,Away_GK_Ability,Home_team_offensive_ability,Away_team_offensive_ability,Home_team_possession_ability,Away_team_possession_ability,Home_team_defensive_ability,Away_team_defensive_ability,Fouls_Difference,Yellows_Difference,Reds_Difference,Referee_Bias_Index,HomeTeam_Elo_Before_Match,AwayTeam_Elo_Before_Match,HomeTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_HomeWinRate,AwayTeam_AwayWinRate,HomeTeam_HomeGoalsRatio,AwayTeam_AwayGoalsRatio,HomeTeam_HomeWinsRatio,AwayTeam_AwayWinsRatio,EPL,HomeTeam_CupGamesLastMonth,AwayTeam_CupGamesLastMonth,HomeTeam_Morale,AwayTeam_Morale,Curse_Parameter,Year,Month,Day
0,Charlton,Man City,H,2.0,0.0,H,17.0,8.0,14.0,4.0,6.0,6.0,13.0,12.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,173.216250,307.286550,2073.122369,774.473532,1686.839900,584.415044,1679.712000,1970.531638,1.0,-1.0,0.0,0.000000,1500.0,1500.0,0.235294,0.000000,0.285714,0.000000,0.134048,0.108753,0.230415,0.235632,0.121277,0.139785,0.257919,0.292793,0.578947,0.210526,0.620000,0.512195,0.785714,0.500000,1.0,0.0,0.0,0.0,0.0,0.0,2000,8,19
1,Chelsea,West Ham,H,1.0,0.0,H,17.0,12.0,10.0,5.0,7.0,7.0,19.0,14.0,1.0,2.0,0.0,0.0,20.5,13.7,16.9,2.6,0.0,16.0,1,136.056680,135.192982,1691.299854,1242.118239,1704.834412,895.031110,901.836801,832.980702,5.0,-1.0,0.0,1.333333,1500.0,1500.0,0.235294,0.166667,0.400000,0.400000,0.134921,0.094737,0.283333,0.196507,0.122616,0.105708,0.252809,0.228311,0.684211,0.210526,0.647059,0.466667,0.764706,0.400000,1.0,0.0,0.0,0.0,0.0,0.0,2000,8,19
2,Coventry,Middlesbrough,A,1.0,1.0,D,6.0,16.0,3.0,9.0,8.0,4.0,15.0,21.0,5.0,3.0,1.0,0.0,18.0,11.4,14.4,0.9,0.0,17.7,1,191.449539,133.710790,1492.648105,1824.347684,1095.830300,1366.709700,1856.523790,1296.619790,-6.0,2.0,1.0,-1.000000,1500.0,1500.0,0.166667,0.187500,0.333333,0.333333,0.092545,0.125000,0.203390,0.263473,0.130435,0.093418,0.253012,0.196429,0.210526,0.263158,0.388889,0.590909,0.500000,0.555556,1.0,0.0,0.0,0.0,0.0,0.0,2000,8,19
3,Derby,Southampton,D,1.0,2.0,A,6.0,13.0,4.0,6.0,5.0,8.0,11.0,13.0,1.0,1.0,0.0,0.0,17.7,9.3,13.7,2.8,0.0,18.6,1,179.294013,145.866316,1534.110553,1658.497895,1366.709700,1686.839900,1738.649263,1414.494316,-2.0,0.0,0.0,-0.666667,1500.0,1500.0,0.333333,0.153846,0.500000,0.333333,0.121311,0.096618,0.262411,0.229885,0.123950,0.118227,0.232283,0.238806,0.421053,0.157895,0.621622,0.325000,0.800000,0.214286,1.0,0.0,0.0,0.0,0.0,0.0,2000,8,19
4,Leeds,Everton,H,2.0,0.0,H,17.0,12.0,8.0,6.0,6.0,4.0,21.0,20.0,1.0,3.0,0.0,0.0,18.0,10.1,13.6,4.0,0.0,20.8,1,130.671908,237.180000,2653.596631,1448.825210,2203.973300,1548.376343,1267.151157,1459.607832,1.0,-2.0,0.0,-0.333333,1500.0,1500.0,0.117647,0.000000,0.250000,0.000000,0.134737,0.114213,0.280702,0.230769,0.111399,0.153247,0.218274,0.308901,0.578947,0.263158,0.562500,0.355556,0.550000,0.454545,1.0,0.0,0.0,0.0,0.0,0.0,2000,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11897,Chelsea,Wolves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [54]:
X = df_combined.drop('FTR', axis=1)  # Features

In [55]:
categorical_features = ['HomeTeam', 'AwayTeam']
numerical_features = X.columns.difference(categorical_features)

# Define ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X_processed = preprocessor.fit_transform(X)
if isinstance(X_processed, csr_matrix):
    X_processed = X_processed.toarray()

In [56]:
ohe = preprocessor.named_transformers_['cat']
ohe_feature_names = ohe.get_feature_names_out(input_features=categorical_features)

# Combine all feature names
all_feature_names = numerical_features.tolist() + list(ohe_feature_names)

# Create a DataFrame from the processed data
X_processed_df = pd.DataFrame(X_processed, columns=all_feature_names)
X_processed_df

,AC,AF,AR,AS,AST,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Day,EPL,Fouls_Difference,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Month,Reds_Difference,Referee_Bias_Index,Season,Year,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h),HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,HTR_A,HTR_D,HTR_H,HTR_nan
0,0.450765,0.048140,-0.296049,-0.584880,-0.259870,0.186252,1.065248,-0.482490,1.014285,-0.574871,-0.569887,-0.013977,-0.950768,-0.970046,1.468170,0.506597,-0.051535,-0.324886,2.860496,1.492118,-1.668073,-1.645181,0.036759,0.332928,0.0,0.307376,-0.019457,0.444840,-0.247135,0.655026,2.413149,-0.703971,1.575055,-0.362381,-0.575018,-0.546592,0.680767,0.648226,1.298574,-0.048776,1.085325,0.049489,0.526663,-0.021380,1.105986,-0.402886,-0.029342,0.798994,1.064309,0.768845,0.327866,0.064684,0.149929,-1.657957,-1.723298,-0.425952,0.092841,-0.121773,1.685755,1.722526,1.742203,-0.777364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.8

In [57]:
mice_imputer = IterativeImputer()
X_processed_df_all_columns = mice_imputer.fit_transform(X_processed_df)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [58]:
# year_2024_df = X_processed_df[X_processed_df['Season'] == 1.762304]
# year_2024_df
X_processed_df = pd.DataFrame(X_processed_df_all_columns, columns=all_feature_names)
X_processed_df

,AC,AF,AR,AS,AST,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Day,EPL,Fouls_Difference,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Month,Reds_Difference,Referee_Bias_Index,Season,Year,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h),HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,HTR_A,HTR_D,HTR_H,HTR_nan
0,0.450765,0.048140,-0.296049,-0.584880,-0.259870,0.186252,1.065248,-0.482490,1.014285,-0.574871,-0.569887,-0.013977,-0.950768,-0.970046,1.468170,0.506597,-0.051535,-0.324886,2.860496,1.492118,-1.668073,-1.645181,0.036759,0.332928,0.0,0.307376,-0.019457,0.444840,-0.247135,0.655026,2.413149,-0.703971,1.575055,-0.362381,-0.575018,-0.546592,0.680767,0.648226,1.298574,-0.048776,1.085325,0.049489,0.526663,-0.021380,1.105986,-0.402886,-0.029342,0.798994,1.064309,0.768845,0.327866,0.064684,0.149929,-1.657957,-1.723298,-0.425952,0.092841,-0.121773,1.685755,1.722526,1.742203,-0.777364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.8

In [61]:
test_df = X_processed_df.tail(10)
test_df

,AC,AF,AR,AS,AST,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Day,EPL,Fouls_Difference,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Month,Reds_Difference,Referee_Bias_Index,Season,Year,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h),HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,HTR_A,HTR_D,HTR_H,HTR_nan
8840,0.081796,0.001954,-0.003497,0.391376,-0.266976,-0.00021,-1.216354,-1.125798,-1.187954,0.408827,-0.501253,-0.545340,-0.284100,0.047911,-0.031903,1.078870,-0.582510,0.778416,0.261851,-0.240227,-0.357000,-0.114448,-0.190251,-1.43225,0.0,-0.001471,-0.140495,0.000138,-0.009692,0.009577,-0.345019,0.102058,-0.010898,0.004603,0.277162,-0.447793,-0.051628,-0.613850,0.044741,-0.658779,-0.031554,0.146780,0.637991,1.628165,-0.332669,0.657981,0.319268,-0.391660,-0.551576,-0.715443,-1.2105,-0.003294,-0.000527,1.762304,1.830998,0.003735,0.007749,0.049405,0.160190,0.087727,0.044245,0.189092,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8

In [62]:
train_df = X_processed_df.iloc[:-10]
train_df

,AC,AF,AR,AS,AST,AY,AwayTeam_AwayGoalsRatio,AwayTeam_AwayWinRate,AwayTeam_AwayWinsRatio,AwayTeam_CupGamesLastMonth,AwayTeam_Elo_Before_Match,AwayTeam_Morale,AwayTeam_Offensive_Shooting_Efficiency,AwayTeam_Offensive_Shots_on_Target_Efficiency,AwayTeam_Season_Defensive_Shooting_Efficiency,AwayTeam_Season_Defensive_Shots_on_Target_Efficiency,AwayTeam_Season_Offensive_Shooting_Efficiency,AwayTeam_Season_Offensive_Shots_on_Target_Efficiency,Away_GK_Ability,Away_team_defensive_ability,Away_team_offensive_ability,Away_team_possession_ability,Curse_Parameter,Day,EPL,Fouls_Difference,HC,HF,HR,HS,HST,HTAG,HTHG,HY,HomeTeam_CupGamesLastMonth,HomeTeam_Elo_Before_Match,HomeTeam_HomeGoalsRatio,HomeTeam_HomeWinRate,HomeTeam_HomeWinsRatio,HomeTeam_Morale,HomeTeam_Offensive_Shooting_Efficiency,HomeTeam_Offensive_Shots_on_Target_Efficiency,HomeTeam_Season_Defensive_Shooting_Efficiency,HomeTeam_Season_Defensive_Shots_on_Target_Efficiency,HomeTeam_Season_Offensive_Shooting_Efficiency,HomeTeam_Season_Offensive_Shots_on_Target_Efficiency,Home_GK_Ability,Home_team_defensive_ability,Home_team_offensive_ability,Home_team_possession_ability,Month,Reds_Difference,Referee_Bias_Index,Season,Year,Yellows_Difference,rain_sum (mm),snowfall_sum (cm),temperature_2m_max,temperature_2m_mean,temperature_2m_min,wind_speed_10m_max (km/h),HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Birmingham,HomeTeam_Blackburn,HomeTeam_Blackpool,HomeTeam_Bolton,HomeTeam_Bournemouth,HomeTeam_Bradford,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Cardiff,HomeTeam_Charlton,HomeTeam_Chelsea,HomeTeam_Coventry,HomeTeam_Crystal Palace,HomeTeam_Derby,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Huddersfield,HomeTeam_Hull,HomeTeam_Ipswich,HomeTeam_Leeds,HomeTeam_Leicester,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Middlesbrough,HomeTeam_Newcastle,HomeTeam_Norwich,HomeTeam_Nott'm Forest,HomeTeam_Portsmouth,HomeTeam_QPR,HomeTeam_Reading,HomeTeam_Sheffield United,HomeTeam_Southampton,HomeTeam_Stoke,HomeTeam_Sunderland,HomeTeam_Swansea,HomeTeam_Tottenham,HomeTeam_Watford,HomeTeam_West Brom,HomeTeam_West Ham,HomeTeam_Wigan,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Birmingham,AwayTeam_Blackburn,AwayTeam_Blackpool,AwayTeam_Bolton,AwayTeam_Bournemouth,AwayTeam_Bradford,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Cardiff,AwayTeam_Charlton,AwayTeam_Chelsea,AwayTeam_Coventry,AwayTeam_Crystal Palace,AwayTeam_Derby,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Huddersfield,AwayTeam_Hull,AwayTeam_Ipswich,AwayTeam_Leeds,AwayTeam_Leicester,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Middlesbrough,AwayTeam_Newcastle,AwayTeam_Norwich,AwayTeam_Nott'm Forest,AwayTeam_Portsmouth,AwayTeam_QPR,AwayTeam_Reading,AwayTeam_Sheffield United,AwayTeam_Southampton,AwayTeam_Stoke,AwayTeam_Sunderland,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,HTR_A,HTR_D,HTR_H,HTR_nan
0,0.450765,0.048140,-0.296049,-0.584880,-0.259870,0.186252,1.065248,-0.482490,1.014285,-0.574871,-0.569887,-0.013977,-0.950768,-0.970046,1.468170,0.506597,-0.051535,-0.324886,2.860496,1.492118,-1.668073,-1.645181,0.036759,0.332928,0.0,0.307376,-0.019457,0.444840,-0.247135,0.655026,2.413149,-0.703971,1.575055,-0.362381,-0.575018,-0.546592,0.680767,0.648226,1.298574,-0.048776,1.085325,0.049489,0.526663,-0.021380,1.105986,-0.402886,-0.029342,0.798994,1.064309,0.768845,0.327866,0.064684,0.149929,-1.657957,-1.723298,-0.425952,0.092841,-0.121773,1.685755,1.722526,1.742203,-0.777364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.8